In [ ]:
import requests
import time
import string
import json
from collections import deque

BASE_URL = "http://35.200.185.69:8000/v1/autocomplete"

MAX_RESULTS_PER_QUERY = 10
REQUEST_DELAY = 0.6
RETRY_DELAY = 5      
SAVE_INTERVAL = 100

seenNames = set()
visitedWords = set()
count = 0

RESULTS_FILE = "v1_names.json"

try:
    with open(RESULTS_FILE, "r") as f:
        seenNames = set(json.load(f))
        print(f"Loaded {len(seenNames)} names from previous run.")
except FileNotFoundError:
    pass


def search_api(prefix):
    global count
    while True:
        try:
            response = requests.get(BASE_URL, params={"query": prefix})
            count += 1

            if response.status_code == 429:
                print("Rate limit exceeded. Sleeping before retrying...")
                time.sleep(RETRY_DELAY)
                continue

            response.raise_for_status()
            data = response.json()
            return data.get("results", [])
        except Exception as e:
            print(f"Error: {e}. Retrying in {RETRY_DELAY} seconds...")
            time.sleep(RETRY_DELAY)


def mainFunction():
    queue = deque(string.ascii_lowercase)

    while queue:
        prefix = queue.popleft()
        if prefix in visitedWords:
            continue
        visitedWords.add(prefix)

        results = search_api(prefix)
        time.sleep(REQUEST_DELAY)

        newNames = [name for name in results if name not in seenNames]
        seenNames.update(newNames)

        if len(results) == MAX_RESULTS_PER_QUERY:
            for c in string.ascii_lowercase:
                new_prefix = prefix + c
                if new_prefix not in visitedWords:
                    queue.append(new_prefix)

        if count % SAVE_INTERVAL == 0:
            save_results()

    save_results()


def save_results():
    with open(RESULTS_FILE, "w") as f:
        json.dump(sorted(seenNames), f, indent=2)
    print(f"Saved {len(seenNames)} names after {count} requests.")


if __name__ == "__main__":
    mainFunction()

Saved 969 names after 100 requests.
Saved 1921 names after 200 requests.
Saved 2876 names after 300 requests.
Saved 3830 names after 400 requests.
Saved 4776 names after 500 requests.
Saved 5734 names after 600 requests.
Saved 6700 names after 700 requests.
Saved 6751 names after 800 requests.
Saved 6775 names after 900 requests.
Saved 6804 names after 1000 requests.
Saved 6835 names after 1100 requests.
Saved 6859 names after 1200 requests.
Saved 6882 names after 1300 requests.
Saved 6898 names after 1400 requests.
Saved 6926 names after 1500 requests.
Saved 6955 names after 1600 requests.
Saved 6977 names after 1700 requests.
Saved 7012 names after 1800 requests.
Saved 7038 names after 1900 requests.
Saved 7061 names after 2000 requests.
Saved 7086 names after 2100 requests.
Saved 7119 names after 2200 requests.
Saved 7143 names after 2300 requests.
Saved 7164 names after 2400 requests.
Saved 7193 names after 2500 requests.
Saved 7229 names after 2600 requests.
Saved 7256 names after